# Liputan 6
## Klasifikasi jenis berita berdasarkan judul

In [1]:
# install libarary
! pip install pandas
! pip install numpy
! pip install nltk
! pip install Sastrawi
! pip install sklearn


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [98]:
# import library

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
# nltk.download()

In [99]:
# load data

df = pd.read_csv("raw.csv")
df.head()

,Judul,Jenis,User Aktif
0,Penyebab Wanita Gagal Raih Orgasme Saat Bercin...,Health,9
1,6 Potret Perayaan Ulang Tahun Okie Agustina ke...,Hot,7
2,Bacaan Doa Setelah Sholat Tahajud Rasulullah S...,Islami,7
3,Ternyata Pihak-Pihak Ini yang Disebut Punya Bi...,Crypto,6
4,Rekomendasi Film-Film Seru di Bioskop Sepanjan...,Regional,6


In [100]:
# memilih kolom-kolom yang akan di drop 

to_drop = ['User Aktif']

# melakukan drop pada data yang dipilih

df.drop(to_drop, inplace=True, axis=1)

# Hasil penghilenang kolom
df.head()

,Judul,Jenis
0,Penyebab Wanita Gagal Raih Orgasme Saat Bercin...,Health
1,6 Potret Perayaan Ulang Tahun Okie Agustina ke...,Hot
2,Bacaan Doa Setelah Sholat Tahajud Rasulullah S...,Islami
3,Ternyata Pihak-Pihak Ini yang Disebut Punya Bi...,Crypto
4,Rekomendasi Film-Film Seru di Bioskop Sepanjan...,Regional


In [101]:
# melihat statistik dasar 

df.describe()

,Judul,Jenis
count,2286,2286
unique,2099,17
top,Senam Lantai: Pengertian Sejarah Manfaat dan M...,Hot
freq,5,514


In [102]:
# menghilangkan data judul yang sama
df.drop_duplicates(subset="Judul", keep=False, inplace=True)
df.describe()

,Judul,Jenis
count,1961,1961
unique,1961,17
top,Penyebab Wanita Gagal Raih Orgasme Saat Bercin...,Hot
freq,1,416


# Case folding

In [103]:
# mengubah ke lower case
df['Judul'] = df['Judul'].str.lower()
df['Jenis'] = df['Jenis'].str.lower()

df.head()

,Judul,Jenis
0,penyebab wanita gagal raih orgasme saat bercin...,health
1,6 potret perayaan ulang tahun okie agustina ke...,hot
3,ternyata pihak-pihak ini yang disebut punya bi...,crypto
4,rekomendasi film-film seru di bioskop sepanjan...,regional
5,manfaat sholat istikharah tata cara niat doa d...,regional


In [104]:
# menghilangkan karakter special dan numerik
df['Judul'] = df['Judul'].str.replace('\d*', '')
df['Judul'] = df['Judul'].str.replace('[^\w\s]', ' ')
df['Judul'] = df['Judul'].str.strip()
df.head()

C:\Users\AKU\AppData\Local\Temp\ipykernel_6244\2487329922.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Judul'] = df['Judul'].str.replace('\d*', '')
C:\Users\AKU\AppData\Local\Temp\ipykernel_6244\2487329922.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Judul'] = df['Judul'].str.replace('[^\w\s]', ' ')


,Judul,Jenis
0,penyebab wanita gagal raih orgasme saat bercin...,health
1,potret perayaan ulang tahun okie agustina ke ...,hot
3,ternyata pihak pihak ini yang disebut punya bi...,crypto
4,rekomendasi film film seru di bioskop sepanjan...,regional
5,manfaat sholat istikharah tata cara niat doa d...,regional


# Tokenizing and Stemming

In [105]:
# stemming and tokenizing function to sastrawati
def funSastrawati(val):
    stop_factory = ArrayDictionary(StopWordRemoverFactory().get_stop_words())
    str = StopWordRemover(stop_factory)
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    word = stemmer.stem(str.remove(val))
    return word
# Apply stemming and tokenizing
df['Judul'] = df['Judul'].apply(funSastrawati)
df.head()

,Judul,Jenis
0,sebab wanita gagal raih orgasme cinta salah sa...,health
1,potret raya ulang tahun okie agustina rias bik...,hot
3,nyata pihak pihak yang sebut punya bitcoin ban...,crypto
4,rekomendasi film film seru bioskop panjang sep...,regional
5,manfaat sholat istikharah tata cara niat doa w...,regional


In [106]:
df['Judul'] = df['Judul'].astype('string')
df['Jenis'] = df['Jenis'].astype('category')
df.dtypes

Judul      string
Jenis    category
dtype: object

In [107]:
df.to_csv(r'P:\UKDC\Kapital Selekta\program\clean.csv', index=False, header=True)

In [108]:
# Mengetahui jumlah jenis berita
df['Jenis'].value_counts()

hot          416
citizen6     283
bisnis       192
health       172
global       158
showbiz      128
lifestyle    106
bola          93
tekno         75
regional      68
news          67
kabar         63
islami        58
otomotif      35
fashion       22
crypto        15
surabaya      10
Name: Jenis, dtype: int64

In [109]:
df['Label'] = df['Jenis'].map({
    'hot' : 0,
    'citizen6' : 1,
    'bisnis' : 2,
    'health' : 3,
    'global' : 4,
    'showbiz' : 5,
    'lifestyle' : 6,
    'bola' : 7,
    'tekno' : 8,
    'regional' : 9,
    'news' : 10,
    'kabar' : 11,
    'islami' : 12,
    'otomotif' : 13,
    'fashion' : 14,
    'crypto' : 15,
    'surabaya' : 16,
})
df.head()

,Judul,Jenis,Label
0,sebab wanita gagal raih orgasme cinta salah sa...,health,3
1,potret raya ulang tahun okie agustina rias bik...,hot,0
3,nyata pihak pihak yang sebut punya bitcoin ban...,crypto,15
4,rekomendasi film film seru bioskop panjang sep...,regional,9
5,manfaat sholat istikharah tata cara niat doa w...,regional,9


In [110]:
# Spliting data (pemodelan)

X_train, X_test, y_train, y_test = train_test_split(df['Judul'].values, df['Label'].values, test_size=0.2, random_state = 42, stratify=df['Label'].values)

# TF-IDF

In [111]:
# tf-idf
tf = TfidfVectorizer()
tfidf_train = tf.fit_transform(X_train)
tfidf_test = tf.transform(X_test)


# KNN

In [112]:
# implementasi data train kedalam KNN
modelknn = KNeighborsClassifier(n_neighbors=3, weights='distance')
modelknn.fit(tfidf_train,y_train)

KNeighborsClassifier(n_neighbors=3, weights='distance')

In [113]:
# melakuakan test KNN
y_pred = modelknn.predict(tfidf_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.56      0.58      0.57        83
           1       0.23      0.26      0.24        57
           2       0.58      0.50      0.54        38
           3       0.32      0.35      0.33        34
           4       0.24      0.19      0.21        32
           5       0.47      0.54      0.50        26
           6       0.16      0.19      0.17        21
           7       0.56      0.79      0.65        19
           8       0.59      0.67      0.62        15
           9       0.12      0.07      0.09        14
          10       0.50      0.31      0.38        13
          11       0.71      0.92      0.80        13
          12       0.38      0.25      0.30        12
          13       0.00      0.00      0.00         7
          14       0.00      0.00      0.00         4
          15       1.00      0.67      0.80         3
          16       0.00      0.00      0.00         2

    accuracy              

C:\Users\AKU\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AKU\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AKU\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [114]:
X_test[:5]

<StringArray>
['mau buka usaha bimbel modal nol rupiah yuk ikut langkah',
      'bintang piala dunia follower banyak medsos favorit',
   'adu musikalitas tren genre musik band ajar yogyakarta',
   'cara dapat set top box stb tv digital gratis perintah',
                                   'beda pria sunat tidak']
Length: 5, dtype: string

In [115]:
y_pred[:5]

array([2, 7, 0, 8, 1], dtype=int64)

# Test Sample

In [116]:
test = pd.read_csv("train.csv")
test.head()

,Judul
0,Berikut adalah potret para pemenang yang berha...
1,Selain Sandrinna Michelle masih ada sembilan p...
2,Berikut ini daftar pemenang SCTV Awards 2022 l...
3,Berikutini Link Live Streaming SCTV Awards 202...
4,SCTV award 2022 menganugerahkan aktor Cinta Br...


In [117]:
# menghilangkan karakter special dan numerik
test['Judul'] = test['Judul'].str.replace('\d*', '')
test['Judul'] = test['Judul'].str.replace('[^\w\s]', ' ')
test['Judul'] = test['Judul'].str.strip()

# Apply stemming and tokenizing
test['Judul'] = test['Judul'].apply(funSastrawati)
test.head()

C:\Users\AKU\AppData\Local\Temp\ipykernel_6244\1742386158.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test['Judul'] = test['Judul'].str.replace('\d*', '')
C:\Users\AKU\AppData\Local\Temp\ipykernel_6244\1742386158.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test['Judul'] = test['Judul'].str.replace('[^\w\s]', ' ')


,Judul
0,ikut potret menang hasil abadi tim kapanlagi c...
1,selain sandrinna michelle ada sembilan menang ...
2,ikut daftar menang sctv awards lengkap kategor...
3,berikutini link live streaming sctv awards tay...
4,sctv award anugerah aktor cinta brian aktor ut...


In [118]:
# prediksi berita
prediksi = modelknn.predict(tf.transform(test['Judul'].values))
test['Jenis'] = prediksi
test['Jenis'] = test['Jenis'].map({
    0 : 'hot',
    1 : 'citizen6',
    2 : 'bisnis',
    3 : 'health',
    4 : 'global',
    5 : 'showbiz',
    6 : 'lifestyle',
    7 : 'bola',
    8 : 'tekno',
    9 : 'regional',
    10 : 'news',
    11 : 'kabar',
    12 : 'islami',
    13 : 'otomotif',
    14 : 'fashion',
    15 : 'crypto',
    16 : 'surabaya',
})
test

,Judul,Jenis
0,ikut potret menang hasil abadi tim kapanlagi c...,bola
1,selain sandrinna michelle ada sembilan menang ...,hot
2,ikut daftar menang sctv awards lengkap kategor...,regional
3,berikutini link live streaming sctv awards tay...,showbiz
4,sctv award anugerah aktor cinta brian aktor ut...,showbiz
...,...,...
126,dramatis marseille curi poin kandang as monaco...,bola
127,sead kolasinac buat marseille curi tiga poin k...,lifestyle
128,jakarta gol menit akhir sead kolasinac bawa ma...,global
129,marseille hadap as monaco tanding ligue stade ...,news


In [119]:
test.to_csv(r'P:\UKDC\Kapital Selekta\program\result.csv', index=False, header=True)